In [1]:
#packages for utilzing the mutagenesis code
import argparse
import re #regular expressions
from Bio import Entrez, SeqIO
Entrez.email = 'sethfrazer.edu'
from skbio import TabularMSA
from skbio import Protein
from skbio.alignment import global_pairwise_align_protein
from Bio.Align import substitution_matrices
import itertools
#packages for utilizing deepBreaks
import os
import subprocess
from deepBreaks.utils import load_obj
from deepBreaks.preprocessing import read_data
import joblib
import pandas as pd
import itertools

In [2]:
def getAcc(accession, check):
    if accession == "manual":

        if check == 0:
            manual = input("Enter Reference Sequence: ")
        else:
            manual = input("Enter Target Sequence: ")

        return(manual)
    
    if accession == "AncAgnatha":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWKYSALAAYMFFLILIGFPINFLTLFVTFKHKKLRQPLNYILVNLAVADLFMVCFGFTVTFYTAMNGYFVFGPTGCNIEGFFATLGGEVALWSLVVLAIERYIVVCKPMGNFRFASTHAIMGVAFTWIMALACAAPPLFGWSRYIPEGMQCSCGPDYYTLNPKYHNESYVIYMFVVHFLLPVTIIFFTYGRLICTVKEAAAQQQESASTQKAEKEVTRMVIIMVIGFLVCWVPYASVAFYIFTNKGADFSATLMTVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncActino":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWKYSVLAAYMFFLILIGFPINFLTLLVTFKHKKLRQPLNYILVNLAVADLFMVCFGFTVTFYTAMNGYFVFGPTGCAIEGFFATLGGQVALWSLVVLAIERYIVVCKPMGNFRFASTHAIMGIAFTWIMALACAAPPLFGWSRYIPEGMQCSCGPDYYTLNPDYHNESYVMYMFVVHFLLPVTIIFFTYGRLICTVKEAAAQQQESASTQKAEKEVTRMVILMVIGFLVAWVPYASVAFWIFTNKGADFTATLMTVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncTeleost":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWQYKLLAVYMFFLICFGFPINGLTLVVTAQHKKLRQPLNFILVNLAVAGLIMVCFGFTVTFYTAINGYFVFGPTGCAIEGFMATLGGQVALWSLVVLAIERYIVVCKPMGSFKFTSTHAIVGIAFTWIMALSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPDYHNESYVMYMFVVHFFLPVTIIFFTYGSLVCTVKAAAAQQQESASTQKAEKEVTRMVILMVLGFLVAWTPYASVAAWIFFNKGAAFTAQSMAVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncClupeoc":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWQFKLLAVYMFFLICFGFPINGLTLVVTAQHKKLRQPLNFILVNLAVAGTIMVCFGFTVTFYTAINGYFVLGPTGCAIEGFMATLGGQVALWSLVVLAIERYIVVCKPMGSFKFSSSHAIAGIAFTWVMAMSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPEYNNESYVLYMFICHFCLPVTIIFFTYGSLVCTVKAAAAQQQESESTQKAEREVTRMVILMVLGFLVAWTPYASFAAWIFFNKGAAFSAQSMAVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncCyprin1":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWQFKLLAVYMFFLICFGFPINGLTLVVTAQHKKLRQPLNFILVNLAVAGTIMVCFGFTVTFYTAINGYFVLGPTGCAIEGFMATLGGQVALWSLVVLAIERYIVVCKPMGSFKFSSSHAIAGIAFTWVMAMSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPEYNNESYVLYMFICHFCLPVTIIFFTYGSLVCTVKAAAAQQQESESTQKAEREVTRMVILMVLGFLVAWTPYASFAAWIFFNKGAAFSAQSMAVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncCyprin2":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWQFKLLAVYMFFLICLGFPINGLTLVVTAQHKKLRQPLNFILVNLAVAGTIMVCFGFTVTFYTAINGYFVLGPTGCAIEGFMATLGGEVALWSLVVLAIERYIVVCKPMGSFKFSSSHAFAGIAFTWVMAMACAAPPLVGWSRYIPEGMQCSCGPDYYTLNPEYNNESYVLYMFICHFILPVTIIFFTYGRLVCTVKAAAAQQQESESTQKAEREVTRMVILMVLGFLVAWTPYATVAAWIFFNKGAAFSAQFMAVPAFFSKSSALYNPVIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncEutelo1":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWQFKLLAVYMFFLICFGFPINGLTLVVTAQHKKLRQPLNFILVNLAVAGLIMVCFGFTVTFYTALNGYFVFGPLGCAIEGFMATLGGQVALWSLVVLAIERYIVVCKPMGSFKFTATHAAAGVAFTWIMAMSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPGYNNESYVMYMFTCHFCLPVTIIFFTYGSLVCTVKAAAAQQQESASTQKAEREVTRMCILMVLGFLVAWTPYASFAAWIFFNKGAAFTAQSMAVPAFFSKSSALFNPIIYVLMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncEutelo2":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLADPWQFKLLAFYMFFLICTGFPINALTLVVTAQNKKLRQPLNFILVNLAVAGLIMVCFGFTVTIYSALNGYFVLGPLGCAIEGFMATLGGQVALWSLVVLAIERYIVVCKPMGSFKFTATHAAAGVAFTWIMAMSCAAPPLFGWSRYIPEGMQCSCGPDYYTLAPGYNNESYVMYMFTCHFCVPVFIIFFTYGSLVMTVKAAAAQQQDSASTQKAEKEVTRMCILMVLGFLVAWTPYASFAAWIFFNKGAAFTAMSMAIPAFFSKSSALFNPIIYVLMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncEutelo3":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLADPWQFKLLAFYMFFLILTGFPINALTLVVTAQNKKLRQPLNFILVNLAVAGLIMVCFGFTVTIYSCLNGYFSLGPLGCAIEGFMATLGGQVSLWSLVVLAIERYIVVCKPMGSFKFTATHAAAGCAFTWIMAMSCAAPPLLGWSRYIPEGIQVSCGPDYYTLAPGYNNESYVMYMFTCHFCVPVFTIFFTYGSLVMTVKAAAAQQQDSASTQKAEKEVTRMCILMVLGFLLAWTPYASFAAWIFFNKGAAFTAMSMAIPSFFSKSSALFNPIIYILMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncEutelo4":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYMVDPIIYKILAFYMFFLICTGTPINGLTLFVTAQNKKLRQPLNYILVNLAVAGLIMCCFGFTITITSAVNGYFVLGPTFCAIEGFMATLGGEVALWSLVVLAVERYIVVCKPMGSFKFTGTHAAAGVLFTWIMAMACAAPPLFGWSRYIPEGMQCSCGPDYYTLAPGFNNESYVIYMFVVHFFVPVFIIFFTYGSLVMTVKAAAAQQQDSASTQKAEKEVTRMCVLMVMGFLVAWTPYASFAGWIFLNKGAAFTALTAAIPAFFAKSSALYNPVIYVLMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncTetrapod":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWKYSVLCAYMFFLILIGFPINFLTLLVTFKHKKLRQPLNYILVNLAVADLFMVCFGFTVTFYTAMNGYFVFGPTGCAIEGFFATLGGQVALWSLVVLAIERYIVVCKPMGNFRFASTHAIMGIAFTWIMALACAAPPLFGWSRYIPEGMQCSCGPDYYTLNPDYHNESYVMYMFVVHFLLPVTIIFFTYGRLICKVKEAAAQQQESASTQKAEKEVTRMVILMVIGFLVAWVPYASVAFWIFTNKGADFTATLMTVPAFFSKSSALYNPIIYVLLNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncSaurops":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWKYKVVCCYIFFLISTGLPINFLTLLVTFKHKKLRQPLNYILVNLAVADLFMACFGFTVTFYTAWNGYFVFGPIGCAIEGFFATLGGQVALWSLVVLAIERYIVVCKPMGNFRFSATHALMGIAFTWVMAFSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPDYHNESYVLYMFVVHFIIPVVVIFFSYGRLICKVREAAAQQQESASTQKAEKEVTRMVILMVLGFLLAWTPYAVVAFWIFTNKGADFTATLMTVPAFFSKSSSLYNPIIYVLMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "AncSquamat":
        return("MNGTEGINFYVPLSNKTGLVRSPFEYPQYYLAEPWKYKVVCCYIFFLISTGLPINFLTLLVTFKHKKLRQPLNYILVNLAVADLFMACFGFTVTFYTAWNGYFVFGPIGCAIEGFFATLGGQVALWSLVVLAIERYIVVCKPMGNFRFSATHALMGIAFTWFMAFSCAAPPLFGWSRYIPEGMQCSCGPDYYTLNPDYHNESYVLYMFVVHFIIPVVVIFFSYGRLICKVREAAAQQQESASTQKAEKEVTRMVILMVLGFLLAWTPYAVVAFWIFTNKGADFSATLMTVPAFFSKSSSLYNPIIYVLMNKQFRNCMITTICCGKNPFGDEDVSSSVSQSKTEVSSVSSSQVSPA")
    if accession == "CNUCHROD":
        return("MNGTEGQDFYVPMSNKTGVVRSPFEYPQYYLAEPWKFSALAAYMFMLILLGFPINFLTLYVTIQHKKLRTPLNYILLNLAVANLFMVFGGFTTTMYTSMNGYFVFGVTGCYIEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAILGVAFSWIMALACAAPPLFGWSRYIPEGMQCSCGIDYYTLKPEVNNESFVIYMFVVHFMIPLLIIFFCYGNLVCTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWVPYASVAFYIFTNQGSDFGPIFMTIPAFFAKSSAIYNPVIYIVMNKQFRNCMITTLCCGKNPLGDEDTSAGKTETSSVSTSQVSPA")
    if accession == "ORCAROD":
        return("MNGTEGLNFYVPFSNKTGVVRSPFEYPQYYLAEPWQFSVLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVANLFMVFGGFTTTLYTSLHAYFIFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGLALTWIMAMACAAPPLVGWSRYIPEGMQCSCGIDYYTLSPEVNNESFVIYMFVVHFTIPLVIIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVVAFLICWVPYASVAFYIFTHQGSDFGPIFMTIPSFFAKSSSIYNPVIYIMMNKQFRNCMLTTLCCGRNPLGDDEASTTASKTETSQVAPA")
    if accession == "HIPPOROD":
        return("MNGTEGPNFYVPFSNKTGVVRSPFEYPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSMHGYFIFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGLALTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTLKPEVNNESFVIYMFVVHFSIPMVIIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSSVYNPVIYIMMNKQFRNCMLTTLCCGKNPLGDDEASTTASKTETSQVAPA")
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "gb")
        handle.close()
        for i,feature in enumerate(record.features):
            if feature.type=='CDS':
                aa = feature.qualifiers['translation'][0]
    except:
        manual = input(f"Accession# {accession} - Not Recognized by NCBI\nPlease Enter Target Sequence to Continue: ")
        return(manual)
    return(aa)

In [3]:
def getCombos(mut_list, wt_seq, mut_acc_file):
    wt_seq = f'{wt_seq}_'
    x = 1
    mut_combos = []
    while x in range(len(mut_list)+1):
        combinations = list(itertools.combinations(mut_list, x))

        for combo in combinations:
            acc = wt_seq

            check = 1
            for i in combo:
                if check == x:    
                    acc+=i
                else:
                    acc = f'{acc}{i},'
                check+=1
            print(acc)

            mut_combos.append(acc)
        x +=1
    #print(mut_combos)
    #print(len(mut_combos))

    with open(mut_acc_file, 'w') as f:
        for mut_acc in mut_combos:
            f.write(f'{mut_acc}\n')
    
    return(mut_combos)

In [4]:
def getMutAcc_fromCSV(mut_file, acc_name):
    # Replace 'your_file.csv' with the path to your CSV file    
    df = pd.read_csv(mut_file)

    #  Access and print the column headers
    column_headers = list(df.columns)
    print(column_headers)

    base_acc = acc_name
    list_of_accs = []

    for row in df.itertuples():
        current_acc = base_acc
        check = 0
        # Iterate through each column value
        for i in range(1, len(row)):
            if row[i] == 1:
                if check == 0:
                    current_acc += '_' + column_headers[i-1]
                    check+=1
                else:
                    current_acc += ',' + column_headers[i-1]
        
        # Append the constructed string to the list and reset current_acc
        if current_acc != base_acc:
            list_of_accs.append(current_acc)
            
        current_acc = base_acc
    print(list_of_accs)
    return(list_of_accs)

In [5]:
def getMut(mut_combos,output_file, out_format='fasta', allow_wt = True, raccession='NM_001014890.2'):
    switch = 0
    bovine = Protein(getAcc(raccession, switch))
    switch+=1

    try:
        mut_combos = open(mut_combos).readlines()
    except:
        pass


    #Fetch sequences to manipulate and align
    test = 0
    if out_format != 'fasta':
        with open(output_file, 'w') as f:
            f.write(f'Accessions\tMutant_Seqs\n')
    else:
        with open(output_file, 'w') as f:
            f.write('')
    for lines in mut_combos:     
    
        if len(lines.split("_")[0]) == 2:
            try:
                accession = f'{lines.split("_")[0]}_{lines.split("_")[1]}'
                muts = lines.split('_')[2]
            except:
                accession = f'{lines.split("_")[0]}_{lines.split("_")[1]}'
                muts = []

        elif '_' not in lines:
            accession = lines
            muts = []
            wt = Protein(getAcc(accession, switch))

        else:
            accession = f'{lines.split("_")[0]}'
            muts = lines.split('_')[1]

        accession = accession.replace('\n','')
        wt = Protein(getAcc(accession, switch))

        substitution_matrix = substitution_matrices.load("BLOSUM45")

        ##Simple example for testing
        #wt=Protein("ABCDEF")
        #bovine=Protein("ABCDEF")

        alignment, score, start_end_positions = global_pairwise_align_protein(bovine, wt, substitution_matrix=substitution_matrix, gap_open_penalty=20)
        dic = alignment.to_dict()
        aligned_bovine = dic[0]
        aligned_wt = dic[1]

        if test == 0:
            print(f'This is the aligned bovine:\n{aligned_bovine}')
            print(f'This is {accession} aligned to bovine:\n{aligned_wt}')
        
        #Check if any mutation is entered
        full_mutation = muts
        if len(muts) == 0:
            if allow_wt == True:
                print("You've requested a WT Sequence")
                if out_format == 'fasta':
                    with open(output_file, 'a') as f:
                        f.write(f'>{accession}\n{wt}\n')

                else:
                    with open(output_file, 'a') as f:
                        f.write(f'{accession}\t{wt}\n')
            else:
                raise Exception("ERROR: Expecting mutation name in the format of XaY where X=old amino acid a=number Y=new amino acid, but received WT sequence")
        else: 
            #Check if single or multiple mutations entered
            print(f'This is the set of mutations: {muts}')

            if "," in muts :
                mutations = muts.split(",")
            else :
                #declare empty list and append single mutation so later can be used as list if only one mutation
                mutations = []
                mutations.append(muts)

            mutated = aligned_wt #need to copy original sequence and accumulate mutations
            for mutation in mutations:
                print(f'Checking mutation: {mutation}')
                #check that mutation is formatted properly
                MutationRegex = re.compile(r'([A-Z])(\d+)([A-Z])')
                if re.match(MutationRegex, mutation):
                    mu = MutationRegex.search(mutation)
                    #print(f'Here is the RegEx results: {mu}')
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                elif 'del' in mutation:
                    MutationRegex = re.compile(r'([A-Z])(\d+)([a-z]+)')
                    mu = MutationRegex.search(mutation)
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                elif 'ins' in mutation:
                    MutationRegex = re.compile(r'([a-z]+)(\d+)([A-Z])')
                    mu = MutationRegex.search(mutation)
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                else :
                    raise Exception("ERROR: Expecting mutation name in the format of XaY where X=old amino acid a=number Y=new amino acid")

                #substract one to account for index starting at zero
                #print(mutsite)
                mutsite = mutsite-1
                #print(mutsite)

                #Count gaps to not count them when finding original site to change
                gaps = aligned_bovine[:mutsite].count('-')
                #print(gaps)


                if new == 'del':
                    new = ''
                if old == 'ins':
                    mutated = str(mutated[:mutsite]) + new + str(mutated[mutsite:])
                else:
                    #match = str(mutated[mutsite+gaps])
                    match = str(mutated[mutsite])

                    #print(match)
                    if match != old :
                        try: 
                            match = str(mutated[mutsite+gaps])
                            if match != old: 
                                    raise Exception()
                            else:
                                mutated = str(mutated[:mutsite+gaps]) + new + str(mutated[mutsite+gaps+1:]) 
                        except:
                            print(f'The AA at site {mutsite+1} is supposed to be {old} but appears to be {match}')
                            raise Exception("ERROR: The mutation label does not match the sequence")
                    else: 
                        mutated = str(mutated[:mutsite]) + new + str(mutated[mutsite+1:]) 

            mutated = mutated.replace('-','')
            mutated.replace('\n','')
            newacc = accession + "_" + full_mutation
            newacc = newacc.replace('\n','')

            if out_format == 'fasta':
                with open(output_file, 'a') as f:
                    f.write(f'>{newacc}\n{mutated}\n')
                    print(newacc, end='\t\t')
                    print(mutated)
            else:
                with open(output_file, 'a') as f:
                    f.write(f'{newacc}\t{mutated}\n')
                    print(newacc, end='\t\t')
                    print(mutated)

    return(newacc , mutated)

In [7]:
#mut_acc_list = getMutAcc_fromCSV('human_rho_mutatnts.csv', 'AncBoreotheria')
raccession = 'NM_001014890.2'
input_file = 'temp.txt'
output_file = 'temp.tsv'
getMut(mut_combos=input_file,output_file=output_file,out_format='tsv',allow_wt=True,raccession=raccession)

c:\Users\safra\anaconda3\envs\deepBreaks\lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


This is the aligned bovine:
MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA-----
This is NM_001085652.1 aligned to bovine:
--MLEEEDFYL---FKNVSNVSPFDGPQYHIAPKWAFTLQAIFMGMVFLIGTPLNFIVLLVTIKYKKLRQPLNYILVNITVGGFLMCIFSIFPVFVSSSQGYFFFGRIACSIDAFVGTLTGLVTGWSLAFLAFERYIVICKPMGNFNFSSSHALAVVICTWIIGIVVSVPPFLGWSRYMPEGLQCSCGPDWYTVGTKYRSEYYTWFIFIFCFVIPLSLICFSYGRLLGALRAVAAQQQESASTQKAEREVSRMVIFMVGSFCLCYVPYAAMAMYMVTNRNHGLDLRLVTIPAFFSKSSCVYNPIIYSFMNKQFRGCIMETVC-GRPMSDDSSVSSTSQRTEVSTVSSSQVSPA
This is the set of mutations: S90D

Checking mutation: S90D

NM_001085652.1_S90D		MLEEEDFYLFKNVSNVSPFDGPQYHIAPKWAFTLQAIFMGMVFLIGTPLNFIVLLVTIKYKKLRQPLNYILVNITVGGFLMCIFDIFPVFVSSSQGYFFFGRIACSIDAFVGTLTGLVTGWSLAFLAFERYIVICKPMGN

c:\Users\safra\anaconda3\envs\deepBreaks\lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


This is the aligned bovine:
MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA-----
This is NM_001085652.1 aligned to bovine:
--MLEEEDFYL---FKNVSNVSPFDGPQYHIAPKWAFTLQAIFMGMVFLIGTPLNFIVLLVTIKYKKLRQPLNYILVNITVGGFLMCIFSIFPVFVSSSQGYFFFGRIACSIDAFVGTLTGLVTGWSLAFLAFERYIVICKPMGNFNFSSSHALAVVICTWIIGIVVSVPPFLGWSRYMPEGLQCSCGPDWYTVGTKYRSEYYTWFIFIFCFVIPLSLICFSYGRLLGALRAVAAQQQESASTQKAEREVSRMVIFMVGSFCLCYVPYAAMAMYMVTNRNHGLDLRLVTIPAFFSKSSCVYNPIIYSFMNKQFRGCIMETVC-GRPMSDDSSVSSTSQRTEVSTVSSSQVSPA
This is the set of mutations: D113A

Checking mutation: D113A

NM_001085652.1_D113A		MLEEEDFYLFKNVSNVSPFDGPQYHIAPKWAFTLQAIFMGMVFLIGTPLNFIVLLVTIKYKKLRQPLNYILVNITVGGFLMCIFSIFPVFVSSSQGYFFFGRIACSIAAFVGTLTGLVTGWSLAFLAFERYIVICKP

c:\Users\safra\anaconda3\envs\deepBreaks\lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


...

c:\Users\safra\anaconda3\envs\deepBreaks\lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


This is the aligned bovine:
MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA-----
This is XM_009283802 aligned to bovine:
MNGTEGQDFYVPMSNKTGVVRSPFEYPQYYLADPWKFSALAAYMFMLILLGFPVNFLTLYVTIQHKKLRTPLNYILLNLAVADLFMVLGGFTTTMYTSMNGYFVFGVTGCYIEGFCATLGGEVGLWSLVVLAIERYIVVCKPMSNFRFGENHAIMGVAFSWVMAFACATPPLFGWSRYIPEGMQCSCGIDYYTPKPEINNESFVIYMFVVHFTIPLTIIFFCYGNLVCTVKEAAAQQQESATTQKAEKEVTRMVIIMVISFLMCWVPYASVAIYIFTNQGADFGPIFMTIPAFFAKSSAIYNPVIYIVMNKQFRNCMITTLCCGKNPLGDEDTSAVKTETSTSSVSTSQVSPA
This is the set of mutations: T217M

Checking mutation: T217M

XM_009283802_T217M		MNGTEGQDFYVPMSNKTGVVRSPFEYPQYYLADPWKFSALAAYMFMLILLGFPVNFLTLYVTIQHKKLRTPLNYILLNLAVADLFMVLGGFTTTMYTSMNGYFVFGVTGCYIEGFCATLGGEVGLWSLVVLAIERYIVVCK

c:\Users\safra\anaconda3\envs\deepBreaks\lib\site-packages\skbio\alignment\_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


This is the aligned bovine:
MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA
This is AY228452 aligned to bovine:
MNGTEGSNFYVPFSNKTGVVRSPFEFPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTIQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVALTWFMALACAAXXXVGWSRYIPEGLQCSCGIDYYTLKPEVNNESFVIYMFVVHFTIPMTVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWVPYASVAFYIFTHQGSNFGPIFMTIPSFFAKSAAIYNPVIYIMMNKQFRNCMIITLCCGKNPLGDDEVSASASKTETSQVAPA
This is the set of mutations: T217I
Checking mutation: T217I
AY228452_T217I		MNGTEGSNFYVPFSNKTGVVRSPFEFPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTIQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVALTW

('AY228452_T217I',
 'MNGTEGSNFYVPFSNKTGVVRSPFEFPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTIQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVALTWFMALACAAXXXVGWSRYIPEGLQCSCGIDYYTLKPEVNNESFVIYMFVVHFTIPMIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWVPYASVAFYIFTHQGSNFGPIFMTIPSFFAKSAAIYNPVIYIMMNKQFRNCMIITLCCGKNPLGDDEVSASASKTETSQVAPA')

In [14]:
wds_mut_list = ['A132S', 'N151S', 'G174D', 'P180R', 'F228L', 'F287L']
wt = 'NM_001280659'
mut_acc_file = 'dolphin_muts_csts.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dolphin_muts_csts.fasta'
out_format = 'fasta'
combos = getCombos(wds_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file, out_format)

NM_001280659_A132S
NM_001280659_N151S
NM_001280659_G174D
NM_001280659_P180R
NM_001280659_F228L
NM_001280659_F287L
NM_001280659_A132S,N151S
NM_001280659_A132S,G174D
NM_001280659_A132S,P180R
NM_001280659_A132S,F228L
NM_001280659_A132S,F287L
NM_001280659_N151S,G174D
NM_001280659_N151S,P180R
NM_001280659_N151S,F228L
NM_001280659_N151S,F287L
NM_001280659_G174D,P180R
NM_001280659_G174D,F228L
NM_001280659_G174D,F287L
NM_001280659_P180R,F228L
NM_001280659_P180R,F287L
NM_001280659_F228L,F287L
NM_001280659_A132S,N151S,G174D
NM_001280659_A132S,N151S,P180R
NM_001280659_A132S,N151S,F228L
NM_001280659_A132S,N151S,F287L
NM_001280659_A132S,G174D,P180R
NM_001280659_A132S,G174D,F228L
NM_001280659_A132S,G174D,F287L
NM_001280659_A132S,P180R,F228L
NM_001280659_A132S,P180R,F287L
NM_001280659_A132S,F228L,F287L
NM_001280659_N151S,G174D,P180R
NM_001280659_N151S,G174D,F228L
NM_001280659_N151S,G174D,F287L
NM_001280659_N151S,P180R,F228L
NM_001280659_N151S,P180R,F287L
NM_001280659_N151S,F228L,F287L
NM_001280659_G1

('NM_001280659_A132S,N151S,G174D,P180R,F228L,F287L',
 'MNGTEGLNFYVPFSNKTGVVRSPFEYPQYYLAEPWQFSVLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVANLFMVFGGFTTTLYTSLHAYFVFGPTGCNLEGFFATLGGEIALWSLVVLSIERYVVVCKPMSNFRFGESHAIMGLALTWIMAMACAAAPLVDWSRYIREGMQCSCGIDYYTSRQEVNNESFVIYMFVVHFTIPLVIIFFCYGQLVLTVKEAAAQQQESATTQKAEKEVTRMVIIMVVAFLICWVPYASVAFYIFTHQGSDFGPILMTIPSFFAKSSSIYNPVIYIMMNKQFRNCMLTTLCCGRNPLGDDEASTTASKTETSQVAPA')

In [ ]:
wds_ni_mut_list = ['S298A', 'M44I', 'C162I', 'P215C', 'M257E', 'R314F']
wt = 'AB087809'
mut_acc_file = 'dan_rer_ni_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dan_rer_sws_ni_mut_seqs.fasta'
combos = getCombos(wds_ni_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
nmoc_mut_list = ['F46L', 'E181H', 'T116F', 'C264A', 'A269T', 'P303S']
wt = 'AB087809'
mut_acc_file = 'dan_rer_nmoc_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dan_rer_sws_nmoc_mut_seqs.fasta'
combos = getCombos(nmoc_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)